In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')

# Set style untuk plotting
plt.style.use('default')
sns.set_palette("husl")

In [2]:
# Fungsi helper untuk basic info
def basic_info(df, filename):
    print(f"\n📊 {filename}")
    print("-" * 40)
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"Data types:\n{df.dtypes}")
    print(f"Missing values:\n{df.isnull().sum()}")
    print(f"Memory usage: {df.memory_usage().sum() / 1024:.2f} KB")
    return df.head()

In [3]:
datasets = {}
file_names = {
    'absensi': 'MineToday Dataset/train/train_absensi.csv',
    'mini_project': 'MineToday Dataset/train/train_mini_project.csv', 
    'pendaftaran': 'MineToday Dataset/train/train_pendaftaran.csv',
    'pretest_ml': 'MineToday Dataset/train/train_pretest_ml.csv',
    'pretest_py': 'MineToday Dataset/train/train_pretest_py.csv',
    'pretest_st': 'MineToday Dataset/train/train_pretest_st.csv',
    'weekly_quiz': 'MineToday Dataset/train/train_weekly_quiz.csv'
}

In [4]:
def load_datasets():
    datasets = {}
    for name, path in file_names.items():
        try:
            datasets[name] = pd.read_csv(path)
            print(f"✅ {name}: {datasets[name].shape}")
        except FileNotFoundError:
            print(f"❌ {name}: File not found at {path}")
        except Exception as e:
            print(f"❌ {name}: Error loading - {e}")
    return datasets

In [5]:
# Analisis detail setiap dataset
for file_name, df in datasets.items():
    print(f"\n{'='*60}")
    sample_data = basic_info(df, file_name)
    print(f"\nSample data (first 3 rows):")
    print(sample_data.head(3))
    
    # Cek unique values untuk kolom kategorikal
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"\n🏷️  Categorical columns unique values:")
        for col in categorical_cols[:5]:  # Limit to first 5 columns
            unique_vals = df[col].nunique()
            print(f"  {col}: {unique_vals} unique values")
            if unique_vals <= 10:
                print(f"    Values: {df[col].unique()[:10]}")

In [6]:
# Analisis ID peserta untuk join datasets
print(f"\n{'='*60}")
print("🔗 PARTICIPANT ID ANALYSIS")
print("-" * 40)

# Cari kolom yang mungkin berisi ID peserta
id_candidates = []
for file_name, df in datasets.items():
    for col in df.columns:
        if any(keyword in col.lower() for keyword in ['id', 'email', 'nama', 'timestamp']):
            id_candidates.append((file_name, col, df[col].nunique()))

print("Potential ID columns:")
for file_name, col, unique_count in id_candidates:
    print(f"  {file_name}: {col} ({unique_count} unique values)")



🔗 PARTICIPANT ID ANALYSIS
----------------------------------------
Potential ID columns:


In [7]:
# Cek timestamp patterns
print(f"\n⏰ TIMESTAMP ANALYSIS")
print("-" * 40)
for file_name, df in datasets.items():
    timestamp_cols = [col for col in df.columns if 'timestamp' in col.lower() or 'tanggal' in col.lower()]
    if timestamp_cols:
        print(f"\n{file_name}:")
        for col in timestamp_cols:
            print(f"  {col}: {df[col].dtype}")
            print(f"    Sample: {df[col].dropna().head(2).tolist()}")



⏰ TIMESTAMP ANALYSIS
----------------------------------------


In [8]:
# Summary statistics untuk numerical columns
print(f"\n{'='*60}")
print("📊 NUMERICAL COLUMNS SUMMARY")
print("-" * 40)

for file_name, df in datasets.items():
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    if len(numerical_cols) > 0:
        print(f"\n{file_name}:")
        print(df[numerical_cols].describe().round(2))


📊 NUMERICAL COLUMNS SUMMARY
----------------------------------------


In [9]:
# Pattern analysis untuk potential target creation
print(f"\n{'='*60}")
print("🎯 POTENTIAL TARGET PATTERNS")
print("-" * 40)

# Analisis completion patterns
completion_indicators = []
for file_name, df in datasets.items():
    if 'absensi' in file_name:
        print(f"\n{file_name}:")
        if 'Pertemuan ke' in df.columns or 'Pertemuan ke-' in df.columns:
            pertemuan_col = 'Pertemuan ke' if 'Pertemuan ke' in df.columns else 'Pertemuan ke-'
            print(f"  Column: {pertemuan_col}")
            print(f"  Data type: {df[pertemuan_col].dtype}")
            print(f"  Unique values: {sorted(df[pertemuan_col].dropna().unique())}")
            print(f"  Missing values: {df[pertemuan_col].isnull().sum()}")
            
            # Convert to numeric untuk cari max
            try:
                numeric_values = pd.to_numeric(df[pertemuan_col], errors='coerce')
                max_pertemuan = numeric_values.max()
                print(f"  Max pertemuan: {max_pertemuan}")
                print(f"  Min pertemuan: {numeric_values.min()}")
                print(f"  Total attendances: {len(df)}")
            except Exception as e:
                print(f"  Error processing pertemuan: {e}")
            
    elif 'quiz' in file_name:
        print(f"\n{file_name}:")
        print(f"  Total quiz records: {df.shape[0]}")
        # Cek jika ada kolom score/nilai
        score_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in ['score', 'nilai', 'point'])]
        if score_cols:
            for col in score_cols[:3]:  # Limit to first 3 score columns
                print(f"  {col}: mean={df[col].mean():.2f}, std={df[col].std():.2f}")
        
    elif 'mini_project' in file_name:
        print(f"\n{file_name}:")
        print(f"  Total project submissions: {df.shape[0]}")
        # Cek jika ada link submissions
        link_cols = [col for col in df.columns if 'link' in col.lower() or 'url' in col.lower()]
        if link_cols:
            for col in link_cols:
                non_empty = df[col].dropna().shape[0]
                print(f"  {col}: {non_empty} non-empty submissions")
        
    elif 'pretest' in file_name:
        test_type = file_name.split('_')[-1].replace('.csv', '').upper()
        print(f"\n{file_name}:")
        print(f"  Pretest {test_type} participants: {df.shape[0]}")
        # Cek score columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            for col in numeric_cols[:3]:  # First 3 numeric columns
                print(f"  {col}: mean={df[col].mean():.2f}, range=[{df[col].min():.1f}, {df[col].max():.1f}]")
    
    elif 'pendaftaran' in file_name:
        print(f"\n{file_name}:")
        print(f"  Total registrations: {df.shape[0]}")
        if 'Status' in df.columns:
            status_counts = df['Status'].value_counts()
            print(f"  Status distribution: {dict(status_counts)}")


🎯 POTENTIAL TARGET PATTERNS
----------------------------------------


In [10]:
print(f"\n📈 CROSS-DATASET ANALYSIS")
print("-" * 40)

# Coba identifikasi common participants
participant_counts = {}
for file_name, df in datasets.items():
    # Cari kolom yang mungkin identifier
    for col in df.columns:
        if 'email' in col.lower() or 'nama' in col.lower():
            unique_participants = df[col].nunique()
            participant_counts[f"{file_name}_{col}"] = unique_participants
            print(f"{file_name} - {col}: {unique_participants} unique participants")

if participant_counts:
    print(f"\nParticipant overlap analysis needed for joining datasets")


📈 CROSS-DATASET ANALYSIS
----------------------------------------


In [11]:
print("🔄 DATA INTEGRATION")
print("="*50)

def parse_score(score_str):
    """Convert '80 / 100' to 0.8"""
    if pd.isna(score_str):
        return np.nan
    if isinstance(score_str, str) and '/' in score_str:
        try:
            num, den = score_str.split('/')
            return float(num.strip()) / float(den.strip())
        except:
            return np.nan
    return score_str

🔄 DATA INTEGRATION


In [12]:
def extract_meeting_number(meeting_str):
    """Extract number from 'Pertemuan 12'"""
    if pd.isna(meeting_str):
        return np.nan
    if isinstance(meeting_str, str):
        match = re.search(r'\d+', meeting_str)
        return int(match.group()) if match else np.nan
    return meeting_str

In [13]:
print("🛠️ FEATURE ENGINEERING")
print("="*50)


🛠️ FEATURE ENGINEERING


In [14]:
def create_attendance_features(df_absensi):
    print("\n📊 Processing Attendance Data...")
    
    # Parse meeting numbers dari 'Pertemuan ke' column
    df_absensi['meeting_num'] = df_absensi['Pertemuan ke'].apply(extract_meeting_number)
    
    # Aggregate per participant
    attendance_stats = df_absensi.groupby('id').agg({
        'meeting_num': ['count', 'max', 'min'],
        'Kualitas materi ': 'mean',
        'Kualitas trainer ': 'mean'
    }).round(3)
    
    # Flatten column names
    attendance_stats.columns = [
        'total_meetings_attended', 'highest_meeting', 'first_meeting',
        'avg_material_quality', 'avg_trainer_quality'
    ]
    
    # Calculate key metrics
    MAX_MEETINGS = 30  # Assuming 30 total meetings
    attendance_stats['attendance_rate'] = (
        attendance_stats['total_meetings_attended'] / MAX_MEETINGS
    ).clip(0, 1).round(3)
    
    # Engagement score (high attendance + good ratings)
    attendance_stats['engagement_score'] = (
        attendance_stats['attendance_rate'] * 0.6 + 
        (attendance_stats['avg_material_quality'] / 5) * 0.2 +
        (attendance_stats['avg_trainer_quality'] / 5) * 0.2
    ).round(3)
    
    return attendance_stats.reset_index()

In [15]:
def create_assessment_features(datasets):
    print("\n📊 Processing Assessment Scores...")
    
    assessment_features = []
    
    # Process each pretest
    for test_name in ['pretest_ml', 'pretest_py', 'pretest_st']:
        df = datasets[test_name].copy()
        
        # Parse scores
        df['score_normalized'] = df['Score'].apply(parse_score)
        
        # Get best attempt per participant
        best_scores = df.groupby('id')['score_normalized'].max().reset_index()
        best_scores.columns = ['id', f'{test_name}_best_score']
        
        assessment_features.append(best_scores)
    
    # Merge all assessment scores
    combined_assessments = assessment_features[0]
    for df in assessment_features[1:]:
        combined_assessments = combined_assessments.merge(df, on='id', how='outer')
    
    # Calculate aggregate metrics
    score_cols = ['pretest_ml_best_score', 'pretest_py_best_score', 'pretest_st_best_score']
    
    combined_assessments['avg_pretest_score'] = (
        combined_assessments[score_cols].mean(axis=1, skipna=True).round(3)
    )
    
    combined_assessments['assessment_completion_rate'] = (
        combined_assessments[score_cols].notna().sum(axis=1) / len(score_cols)
    ).round(3)
    
    # Performance category
    combined_assessments['performance_level'] = pd.cut(
        combined_assessments['avg_pretest_score'], 
        bins=[0, 0.5, 0.7, 0.85, 1.0], 
        labels=['Low', 'Medium', 'High', 'Excellent']
    )
    
    return combined_assessments

In [16]:
def create_submission_features(datasets):
    print("\n📊 Processing Submission Data...")
    
    submission_data = []
    
    # Mini project submissions
    mini_project = datasets['mini_project'].groupby('id').size().reset_index()
    mini_project.columns = ['id', 'mini_project_count']
    mini_project['has_mini_project'] = (mini_project['mini_project_count'] > 0).astype(int)
    submission_data.append(mini_project[['id', 'has_mini_project']])
    
    # Weekly quiz submissions
    weekly_quiz = datasets['weekly_quiz'].groupby('id').size().reset_index()
    weekly_quiz.columns = ['id', 'quiz_submissions']
    weekly_quiz['has_weekly_quiz'] = (weekly_quiz['quiz_submissions'] > 0).astype(int)
    submission_data.append(weekly_quiz[['id', 'has_weekly_quiz']])
    
    # Combine submission features
    combined_submissions = submission_data[0]
    for df in submission_data[1:]:
        combined_submissions = combined_submissions.merge(df, on='id', how='outer')
    
    # Fill missing values
    combined_submissions = combined_submissions.fillna(0)
    
    # Overall submission rate
    combined_submissions['total_submissions'] = (
        combined_submissions['has_mini_project'] + 
        combined_submissions['has_weekly_quiz']
    )
    
    return combined_submissions

In [17]:
def create_registration_features(df_registration):
    print("\n📊 Processing Registration Data...")
    
    reg_features = df_registration[['id', 'Status', 'Pilihan Jadwal Kelas']].copy()
    
    # Encode status
    status_mapping = {
        'Mahasiswa': 1, 'Fresh Graduates': 2, 
        'Pekerja aktif': 3, 'Umum': 4
    }
    reg_features['status_encoded'] = reg_features['Status'].map(status_mapping)
    
    # Extract batch info
    reg_features['batch'] = reg_features['Pilihan Jadwal Kelas'].str.extract(r'Batch (\d+)')
    reg_features['batch'] = pd.to_numeric(reg_features['batch'], errors='coerce')
    
    return reg_features[['id', 'status_encoded', 'batch']]

In [18]:
# Execute feature engineering
datasets = load_datasets()

# Get all unique participant IDs (missing from script 1)
print(f"\n📋 Unique participants per dataset:")
all_participant_ids = set()
for name, df in datasets.items():
    unique_ids = df['id'].nunique()
    print(f"  {name}: {unique_ids} unique IDs")
    all_participant_ids.update(df['id'].unique())

print(f"\n👥 Total unique participants across all datasets: {len(all_participant_ids)}")

# Create master participant table
master_df = pd.DataFrame({'id': list(all_participant_ids)})
print(f"✅ Master participant table created with {len(master_df)} participants")

attendance_features = create_attendance_features(datasets['absensi'])
assessment_features = create_assessment_features(datasets)
submission_features = create_submission_features(datasets)
registration_features = create_registration_features(datasets['pendaftaran'])

✅ absensi: (11714, 12)
✅ mini_project: (468, 5)
✅ pendaftaran: (492, 9)
✅ pretest_ml: (502, 14)
✅ pretest_py: (544, 14)
✅ pretest_st: (500, 19)
✅ weekly_quiz: (487, 5)

📋 Unique participants per dataset:
  absensi: 509 unique IDs
  mini_project: 468 unique IDs
  pendaftaran: 492 unique IDs
  pretest_ml: 494 unique IDs
  pretest_py: 526 unique IDs
  pretest_st: 497 unique IDs
  weekly_quiz: 483 unique IDs

👥 Total unique participants across all datasets: 549
✅ Master participant table created with 549 participants

📊 Processing Attendance Data...

📊 Processing Assessment Scores...

📊 Processing Submission Data...

📊 Processing Registration Data...


In [19]:
print(f"\n🔄 Merging all features...")
final_features = master_df.copy()

# Merge each feature set
feature_sets = [
    ('attendance', attendance_features),
    ('assessment', assessment_features), 
    ('submission', submission_features),
    ('registration', registration_features)
]

for name, features in feature_sets:
    before_count = len(final_features)
    final_features = final_features.merge(features, on='id', how='left')
    after_count = len(final_features)
    print(f"  ✅ Merged {name}: {before_count} → {after_count} rows")

print(f"\n🎯 Final feature matrix: {final_features.shape}")
print(f"Columns: {list(final_features.columns)}")

# Show sample of final features
print(f"\n📊 Sample of engineered features:")
print(final_features.head())


🔄 Merging all features...
  ✅ Merged attendance: 549 → 549 rows
  ✅ Merged assessment: 549 → 549 rows
  ✅ Merged submission: 549 → 549 rows
  ✅ Merged registration: 549 → 549 rows

🎯 Final feature matrix: (549, 19)
Columns: ['id', 'total_meetings_attended', 'highest_meeting', 'first_meeting', 'avg_material_quality', 'avg_trainer_quality', 'attendance_rate', 'engagement_score', 'pretest_ml_best_score', 'pretest_py_best_score', 'pretest_st_best_score', 'avg_pretest_score', 'assessment_completion_rate', 'performance_level', 'has_mini_project', 'has_weekly_quiz', 'total_submissions', 'status_encoded', 'batch']

📊 Sample of engineered features:
                                     id  total_meetings_attended  \
0  366c8261-243e-4eab-9695-6df81979597a                     18.0   
1  84d29b44-26c6-4b05-8a67-31831d35be32                     28.0   
2  e0766991-32fc-4294-b1e2-a0dff52c64f2                     23.0   
3  a59fa924-4733-471c-8754-f6595cd0e6f1                     27.0   
4  ab9d4c1f

In [20]:
print("🎯 TARGET CREATION")
print("="*50)

# Fill missing values untuk target creation
print("\n🔧 Preprocessing for target creation...")

# Select features for target creation
target_features = [
    'attendance_rate', 'highest_meeting', 'total_meetings_attended',
    'avg_pretest_score', 'assessment_completion_rate', 
    'total_submissions', 'engagement_score'
]

# Create a subset with target features
target_df = final_features[['id'] + target_features].copy()

# Impute missing values
imputer = SimpleImputer(strategy='median')
target_df[target_features] = imputer.fit_transform(target_df[target_features])

print(f"✅ Target features prepared for {len(target_df)} participants")

🎯 TARGET CREATION

🔧 Preprocessing for target creation...
✅ Target features prepared for 549 participants


In [21]:
# ============================================================================
# METHOD 1: RULE-BASED LABELING
# ============================================================================

def create_rule_based_labels(df):
    print("\n📋 Method 1: Rule-Based Labeling")
    print("-" * 40)
    
    # Define graduation criteria
    conditions = {
        'high_attendance': df['attendance_rate'] >= 0.75,  # Attended 75%+ meetings
        'reached_advanced': df['highest_meeting'] >= 20,   # Reached meeting 20+
        'good_performance': df['avg_pretest_score'] >= 0.6, # Average score 60%+
        'active_submission': df['total_submissions'] >= 1,  # Submitted at least 1 project
        'high_engagement': df['engagement_score'] >= 0.7   # Overall engagement 70%+
    }
    
    # Calculate completion score
    df['completion_score'] = (
        conditions['high_attendance'].astype(int) * 0.35 +      # Attendance weight
        conditions['reached_advanced'].astype(int) * 0.25 +     # Progress weight  
        conditions['good_performance'].astype(int) * 0.2 +      # Performance weight
        conditions['active_submission'].astype(int) * 0.1 +     # Submission weight
        conditions['high_engagement'].astype(int) * 0.1         # Engagement weight
    )
    
    # Create binary labels
    GRADUATION_THRESHOLD = 0.6  # Need 60% overall score to graduate
    df['graduated_rule'] = (df['completion_score'] >= GRADUATION_THRESHOLD).astype(int)
    
    # Show criteria breakdown
    print("Graduation Criteria:")
    for criterion, condition in conditions.items():
        count = condition.sum()
        pct = count / len(df) * 100
        print(f"  {criterion}: {count} participants ({pct:.1f}%)")
    
    graduate_count = df['graduated_rule'].sum()
    print(f"\n🎓 Rule-based graduation rate: {graduate_count}/{len(df)} ({graduate_count/len(df)*100:.1f}%)")
    
    return df

In [22]:
# ============================================================================
# METHOD 2: CLUSTERING-BASED LABELING  
# ============================================================================

def create_cluster_based_labels(df):
    print("\n🔍 Method 2: Clustering-Based Labeling")
    print("-" * 40)
    
    # Prepare features for clustering
    cluster_features = [
        'attendance_rate', 'avg_pretest_score', 'total_submissions', 
        'engagement_score', 'highest_meeting'
    ]
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[cluster_features])
    
    # Apply K-means clustering (k=3: low, medium, high performers)
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    df['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Analyze clusters to identify high performers
    cluster_analysis = df.groupby('cluster')[cluster_features].mean().round(3)
    print("Cluster Analysis:")
    print(cluster_analysis)
    
    # Identify the "high performer" cluster (highest attendance + performance)
    cluster_scores = cluster_analysis['attendance_rate'] + cluster_analysis['avg_pretest_score']
    high_performer_cluster = cluster_scores.idxmax()
    
    # Create binary labels (high performer cluster = graduated)
    df['graduated_cluster'] = (df['cluster'] == high_performer_cluster).astype(int)
    
    cluster_counts = df['cluster'].value_counts().sort_index()
    print(f"\nCluster distribution:")
    for cluster, count in cluster_counts.items():
        pct = count / len(df) * 100
        status = "HIGH PERFORMER" if cluster == high_performer_cluster else "REGULAR"
        print(f"  Cluster {cluster}: {count} participants ({pct:.1f}%) - {status}")
    
    graduate_count = df['graduated_cluster'].sum()
    print(f"\n🎓 Cluster-based graduation rate: {graduate_count}/{len(df)} ({graduate_count/len(df)*100:.1f}%)")
    
    return df

In [23]:
# ============================================================================
# EXECUTE TARGET CREATION
# ============================================================================

# Create labels using both methods
target_df = create_rule_based_labels(target_df)
target_df = create_cluster_based_labels(target_df)


📋 Method 1: Rule-Based Labeling
----------------------------------------
Graduation Criteria:
  high_attendance: 320 participants (58.3%)
  reached_advanced: 465 participants (84.7%)
  good_performance: 527 participants (96.0%)
  active_submission: 549 participants (100.0%)
  high_engagement: 518 participants (94.4%)

🎓 Rule-based graduation rate: 436/549 (79.4%)

🔍 Method 2: Clustering-Based Labeling
----------------------------------------
Cluster Analysis:
         attendance_rate  avg_pretest_score  total_submissions  \
cluster                                                          
0                  0.909              0.725              1.945   
1                  0.698              0.781              1.975   
2                  0.000              0.784              1.828   

         engagement_score  highest_meeting  
cluster                                     
0                   0.906           27.326  
1                   0.779           21.042  
2                   0.36

In [24]:
# ============================================================================
# LABEL VALIDATION & COMPARISON
# ============================================================================

print("\n🔍 LABEL VALIDATION")
print("="*50)

# Compare the two labeling methods
agreement = (target_df['graduated_rule'] == target_df['graduated_cluster']).sum()
agreement_rate = agreement / len(target_df) * 100

print(f"Agreement between methods: {agreement}/{len(target_df)} ({agreement_rate:.1f}%)")

# Cross-tabulation
crosstab = pd.crosstab(
    target_df['graduated_rule'], 
    target_df['graduated_cluster'], 
    margins=True
)
print(f"\nCross-tabulation:")
print(crosstab)


🔍 LABEL VALIDATION
Agreement between methods: 349/549 (63.6%)

Cross-tabulation:
graduated_cluster    0    1  All
graduated_rule                  
0                  113    0  113
1                  200  236  436
All                313  236  549


In [25]:
# ============================================================================
# FINAL TARGET SELECTION
# ============================================================================

print(f"\n🎯 FINAL TARGET SELECTION")
print("-" * 30)

# Use ensemble approach: agree on both methods OR high completion score
target_df['graduated_final'] = (
    (target_df['graduated_rule'] == 1) & 
    (target_df['graduated_cluster'] == 1)
).astype(int)

# For cases where methods disagree, use completion score as tiebreaker
disagreement_mask = target_df['graduated_rule'] != target_df['graduated_cluster']
high_score_mask = target_df['completion_score'] >= 0.65

target_df.loc[disagreement_mask & high_score_mask, 'graduated_final'] = 1

final_graduate_count = target_df['graduated_final'].sum()
final_rate = final_graduate_count / len(target_df) * 100

print(f"Final graduation rate: {final_graduate_count}/{len(target_df)} ({final_rate:.1f}%)")

# Show final target distribution
print(f"\nFinal Target Distribution:")
print(f"  Graduated (1): {final_graduate_count} participants")
print(f"  Not Graduated (0): {len(target_df) - final_graduate_count} participants")

# Class balance check
if final_rate < 30 or final_rate > 70:
    print(f"⚠️  Class imbalance detected! Consider SMOTE for modeling.")
else:
    print(f"✅ Reasonable class balance for modeling.")

# Merge final target back to feature matrix
final_features_with_target = final_features.merge(
    target_df[['id', 'graduated_final', 'completion_score']], 
    on='id', 
    how='left'
)

print(f"\n🎉 FINAL DATASET READY!")
print(f"Shape: {final_features_with_target.shape}")
print(f"Features: {final_features_with_target.shape[1] - 1}")  # -1 for target
print(f"Target column: 'graduated_final'")

# Save the final dataset
# final_features_with_target.to_csv('bootcamp_final_dataset.csv', index=False)
# print(f"💾 Dataset saved as 'bootcamp_final_dataset.csv'")

# Show sample of final dataset
print(f"\n📊 Final Dataset Sample:")
sample_cols = ['id', 'attendance_rate', 'avg_pretest_score', 'total_submissions', 'graduated_final']
print(final_features_with_target[sample_cols].head(10))


🎯 FINAL TARGET SELECTION
------------------------------
Final graduation rate: 436/549 (79.4%)

Final Target Distribution:
  Graduated (1): 436 participants
  Not Graduated (0): 113 participants
⚠️  Class imbalance detected! Consider SMOTE for modeling.

🎉 FINAL DATASET READY!
Shape: (549, 21)
Features: 20
Target column: 'graduated_final'

📊 Final Dataset Sample:
                                     id  attendance_rate  avg_pretest_score  \
0  366c8261-243e-4eab-9695-6df81979597a            0.600              0.800   
1  84d29b44-26c6-4b05-8a67-31831d35be32            0.933              0.783   
2  e0766991-32fc-4294-b1e2-a0dff52c64f2            0.767              0.767   
3  a59fa924-4733-471c-8754-f6595cd0e6f1            0.900              0.683   
4  ab9d4c1f-1773-4f12-88e1-ecd78f51e309            1.000              0.867   
5  f29b1b86-ab96-4876-889d-70b5955a01c7            0.900              0.600   
6  e62e4b0c-e929-48f4-ad21-f172df57e747            0.767              0.833   
7